In [5]:
import pandas
import psycopg2

In [6]:
"""
connect database with psycopg2
"""

host = 'localhost'
port = '5432'
dbname = 'mydatabase'
username = 'maciek'
pwd = 'dbhaslo'

connection = psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, 
                                                                                         port, 
                                                                                         dbname, 
                                                                                         username, 
                                                                                         pwd))


In [8]:
"""
read database to check if all works
"""
query_interactions = "select * from user_interaction;"
query_users = "select * from users;"
user_interactions_dataframe = pandas.read_sql_query(query_interactions,connection)
users_dataframe = pandas.read_sql_query(query_users,connection)
# print(user_interactions_dataframe)
# print(users_dataframe)